In [1]:
%pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/11.6 MB ? eta -:--:--Downloading transformers-4.56.1-py3-none-any.whl (11.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 2.4 MB/s eta 0:00:0000:010:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 2.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.3 MB ? eta -:--:--Downloading tokenizers-0.22.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3

In [3]:
%pip install 'accelerate>=0.26.0'


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import load_dataset   
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, AutoTokenizer
import numpy as np
# import wandb
import torch

# CHANGE THIS TO YOUR OWN DATASET
dsn = "cubbk/zac_sample-dataset-tokenised"
model_name = "canopylabs/orpheus-tts-0.1-pretrained"

# Training Args
epochs = 1
batch_size = 1
number_processes = 1
pad_token = 128263
save_steps = 5000
learning_rate = 5.0e-5

# Naming and paths
save_folder = "checkpoints"
project_name = "tuning-orpheus"
run_name = "5e5-0"

# Ensure bf16 only when supported
bf16_supported = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
dtype = torch.bfloat16 if bf16_supported else torch.float32
if not bf16_supported:
    print("bfloat16 not supported on this device; using float32.")

# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = pad_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=dtype,
)

# Dataset
ds = load_dataset(dsn, split="train")

# wandb.init(project=project_name, name=run_name)

training_args = TrainingArguments(
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    logging_steps=1,
    bf16=bf16_supported,
    output_dir=f"./{save_folder}",
    # report_to="wandb",
    save_steps=save_steps,
    remove_unused_columns=True,
    learning_rate=learning_rate,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds, # type: ignore
)

trainer.train()


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Step,Training Loss
1,5.142800
2,4.919000
3,4.879400
4,4.782100
5,4.825300
6,4.547100
7,4.501300
8,4.652200
9,4.716300
10,4.566900


TrainOutput(global_step=20, training_loss=4.751743054389953, metrics={'train_runtime': 35.9424, 'train_samples_per_second': 0.556, 'train_steps_per_second': 0.556, 'total_flos': 181673906909184.0, 'train_loss': 4.751743054389953, 'epoch': 1.0})